In [1]:
import numpy as np
import geopandas as gpd


import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from xhistogram.xarray import histogram
import dask.dataframe as dd
#import cartopy.crs as ccrs
#import cartopy.feature as cfeature

import mitequinox.utils as ut
from mitequinox.plot import *
import mitequinox.parcels as pa
from xmitgcm import llcreader

from scipy import signal
import scipy.ndimage as im

from sympy import Symbol, pi, atan, factor, lambdify

import mitequinox.plot as pl
import mitequinox.sigp as sp
import pyinterp

from fsspec.implementations.local import LocalFileSystem

INFO: Compiled ParcelsRandom ==> /dev/shm/pbs.7979979.datarmor0/parcels-502090/libparcels_random_b6129454-24f1-4924-aa4a-d4b3753f2fe0.so


In [2]:
from dask.distributed import Client, LocalCluster
#
#cluster = LocalCluster()
#
from dask_jobqueue import PBSCluster
cluster = PBSCluster() #processes=7, cores=7
w = cluster.scale(jobs=6
                 )
#
client = Client(cluster)

In [3]:
def convolve(x, h=None, hilbert=False):
    """ Convolve an input signal with a kernel
    Optionaly compute the Hilbert transform of the resulting time series
    """
    #x_f = im.convolve1d(x, h, axis=1, mode='constant')
    x_f = signal.filtfilt(h, [1], x, axis=-1,padlen=0)#
    if hilbert:
        return signal.hilbert(x_f)
    else:
        return x_f


In [4]:
root_dir = '/home/datawork-lops-osi/equinox/mit4320/parcels/'
run_name = 'global_extra_T365j_dt1j_dij50'
parcels_index = 'time_mean'

# choose to select time indexed data for now
p = pa.parcels_output(root_dir+run_name, parquets=[parcels_index])
df = p[parcels_index]
df = pa.degs2ms(df)
df.head()

df = df.drop(columns=['temperature','salinity','Eta_interp',
                      'SSU_interp','SSV_interp','SST_interp','SSS_interp']).persist()

In [5]:
dt = 1/24 # in days

tidal_omega = sp.get_tidal_frequencies("M2", "K2","S2","N2","K1","O1","M4","S4")
omega_M2,omega_S2,omega_N2,omega_K2, domega, name = tidal_omega["M2"],tidal_omega["S2"],tidal_omega["N2"],tidal_omega["K2"], .2, "semidiurnal"
omega = (omega_M2+omega_S2)/2

V = ['zonal_velocity','meridional_velocity']
Tw = 30
dband=0.2 #half bandwidth

h = sp.generate_filter(omega, T=Tw, dt=dt, bandwidth=dband, normalized_bandwidth=None)

In [6]:
def remove_duplicates(df):
    df = df.reset_index().drop_duplicates('time').sort_values('time').set_index('time')
    dt_df = df.index[1:]-df.index[:-1]
    df['dt_flag'] = np.insert(dt_df!=np.timedelta64(1,'h'),0,False)
    return df

def filt_L(v, h,columns,om, hilbert=False):
    vc = v.copy()
    #vc = vc.groupby(vc.index).first()#.drop_duplicates
    #vc = vc.groupby(level=df1.index.names).last()
    for V in columns:
        vc[V+'_hat'] = convolve(vc[V],h=h,hilbert=hilbert)
    time = np.arange(0,vc.index.size*dt,dt)#.compute()
    exp = np.exp(-1j*om*2*np.pi*time)
    for V in columns:
        vc[V+'_demodulated'] = vc[V+'_hat']*exp
    return vc

In [7]:
# trajectoire négative (NaN) dans certaines partitions (p.e. partition 0), origine inconnue
#df = df.repartition(npartitions=30)
_df = df.dropna().persist()

In [8]:
df_drop = _df.groupby('trajectory').apply(remove_duplicates
                                   ).drop(columns='trajectory').reset_index().set_index('time').persist()

<ipython-input-8-a411c807b486>:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df_drop = _df.groupby('trajectory').apply(remove_duplicates


In [10]:
df_filtered = df_drop.groupby('trajectory').apply(filt_L,h,['zonal_velocity','meridional_velocity']
                                                  ,omega,hilbert=True).persist()
#df_filtered = df_filtered.repartition(npartitions=10).persist()

<ipython-input-10-cde957a09447>:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df_filtered = df_drop.groupby('trajectory').apply(filt_L,h,['zonal_velocity','meridional_velocity']


In [10]:
df_filtered = df_filtered.drop(columns='trajectory').reset_index().sort_values('time').set_index('time').persist()

In [10]:
for v in ["zonal_velocity","meridional_velocity"]:
    df_filtered[v+"_hat_real"] = df_filtered[v+"_hat"].map_partitions(lambda x : np.real(x)).persist()

    df_filtered[v+"_hat_imag"] = df_filtered[v+"_hat"].map_partitions(lambda x : np.imag(x)).persist()
    df_filtered[v+"_demodulated_real"] = df_filtered[v+"_demodulated"].map_partitions(lambda x : np.real(x)).persist()
    df_filtered[v+"_demodulated_imag"] = df_filtered[v+"_demodulated"].map_partitions(lambda x : np.imag(x)).persist()

In [13]:
for v in ["zonal_velocity","meridional_velocity"]:
    df_filtered = df_filtered.drop(columns=[v+"_hat",v+"_demodulated"]).persist()

In [11]:
time = df_filtered.index.values.compute()
#print(time.size,np.unique(time).size)

KeyboardInterrupt: 

In [15]:
df_filtered.index.values.compute()

OSError: Timed out trying to connect to tcp://10.148.0.223:39661 after 10 s

In [11]:
df_filtered = df_filtered.map_partitions(lambda x: x.loc[np.unique(time)[::6]]).persist()

In [ ]:
time = df_filtered.index.values.compute()

In [22]:
ds, dirs = pa.load_logs(root_dir, run_name)
pa.store_parquet(dirs["parquets"], df_filtered, overwrite=True, name="L_filtered_T30_sdband04")
#p.store_diagnostic("ZC_filteredL_T30_band04", df_filtered,overwrite=True)

No reindexing
deleting existing archive: /home/datawork-lops-osi/equinox/mit4320/parcels/global_extra_T365j_dt1j_dij50/parquets/L_filtered_T30_sdband04
create new archive: /home/datawork-lops-osi/equinox/mit4320/parcels/global_extra_T365j_dt1j_dij50/parquets/L_filtered_T30_sdband04


'/home/datawork-lops-osi/equinox/mit4320/parcels/global_extra_T365j_dt1j_dij50/parquets/L_filtered_T30_sdband04'

In [3]:
root_dir = '/home/datawork-lops-osi/equinox/mit4320/parcels/'
run_name = 'global_extra_T365j_dt1j_dij50'
parcels_index = 'L_filtered_T30_sdband04'

# choose to select time indexed data for now
p = pa.parcels_output(root_dir+run_name, parquets=[parcels_index])
df = p[parcels_index]
df = pa.degs2ms(df)
df.head()

,trajectory,lat,lon,z,zonal_velocity,meridional_velocity,sea_level,dt_flag,zonal_velocity_hat_real,zonal_velocity_hat_imag,zonal_velocity_demodulated_real,zonal_velocity_demodulated_imag,meridional_velocity_hat_real,meridional_velocity_hat_imag,meridional_velocity_demodulated_real,meridional_velocity_demodulated_imag
time,,,,,,,,,,,,,,,,
2011-11-15,2000384,-68.021919,-19.239584,0.0,0.0,0.0,0.0,False,0.004085,0.000697,0.004085,0.000697,0.002623,0.004498,0.002623,0.004498
2011-11-15,48000573,55.309990,-40.197918,0.0,0.0,0.0,0.0,False,0.017263,-0.016716,0.017263,-0.016716,-0.005707,0.016081,-0.005707,0.016081
2011-11-15,48000631,44.620720,-37.072918,0.0,0.0,0.0,0.0,False,0.006110,-0.020936,0.006110,-0.020936,0.027630,-0.014982,0.027630,-0.014982
2011-11-15,49000118,64.051880,-88.307640,0.0,0.0,0.0,0.0,False,-0.180781,-0.079333,-0.180781,-0.079333,-0.485498,0.112855,-0.485498,0.112855
2011-11-15,49000872,64.252319,-36.253830,0.0,0.0,0.0,0.0,False,0.034166,0.012898,0.034166,0.012898,-0.028217,0.020949,-0.028217,0.020949


In [18]:
diags_dir = '/home/datawork-lops-osi/equinox/mit4320/filtered_itide/'
        #if np.isin(v,['SSU','SSV']):
            #ds = xr.merge([ut.load_diagnostic("mean_"+V) for V in ['SSU','SSV']])
#ds = xr.merge([xr.open_zarr(ut.work_data_dir+'rechunked/SSU.zarr'),xr.open_zarr(ut.work_data_dir+'rechunked/SSV.zarr')])#xr.open_zarr(diags_dir+'mean.zarr')[['SSU','SSV']]
ds = xr.open_zarr(diags_dir+'ZC_filtered_itide_face1.zarr')

In [19]:
ds = ds.chunk({'time':1,'i':-1,'j':-1 })

In [20]:
ds

<xarray.Dataset>
Dimensions:          (i: 2160, j: 2160, time: 1464)
Coordinates:
    CS               (j, i) float32 dask.array<chunksize=(2160, 2160), meta=np.ndarray>
    Depth            (j, i) float32 dask.array<chunksize=(2160, 2160), meta=np.ndarray>
    SN               (j, i) float32 dask.array<chunksize=(2160, 2160), meta=np.ndarray>
    XC               (j, i) float32 dask.array<chunksize=(2160, 2160), meta=np.ndarray>
    YC               (j, i) float32 dask.array<chunksize=(2160, 2160), meta=np.ndarray>
  * i                (i) int64 0 2 4 6 8 10 12 ... 4308 4310 4312 4314 4316 4318
  * j                (j) int64 0 2 4 6 8 10 12 ... 4308 4310 4312 4314 4316 4318
    lat              (j, i) float64 dask.array<chunksize=(2160, 2160), meta=np.ndarray>
    lon              (j, i) float64 dask.array<chunksize=(2160, 2160), meta=np.ndarray>
    rA               (j, i) float32 dask.array<chunksize=(2160, 2160), meta=np.ndarray>
  * time             (time) datetime64[ns] 2011-11-15 ... 2012-11-14T18:00:00
Data variables:
    SSU_demodulated  (j, i, time) complex128 dask.array<chunksize=(2160, 2160, 1), meta=np.ndarray>
    SSU_hat          (j, i, time) complex128 dask.array<chunksize=(2160, 2160, 1), meta=np.ndarray>
    SSV_demodulated  (j, i, time) complex128 dask.array<chunksize=(2160, 2160, 1), meta=np.ndarray>
    SSV_hat          (j, i, time) complex128 dask.array<chunksize=(2160, 2160, 1), meta=np.ndarray>
Attributes:
    Tw:         30
    bandwidth:  0.4
    dt:         6h
    face:       1
    omega:      1.9661368071336223

In [21]:
_ds = xr.open_zarr(ut.work_data_dir+'rechunked/SSU.zarr')

In [23]:
_ds

,Array,Chunk
Bytes,1.94 TiB,115.80 MiB
Shape,"(8784, 13, 2160, 2160)","(2196, 1, 96, 144)"
Count,17941 Tasks,17940 Chunks
Type,float32,numpy.ndarray


In [7]:
df[df.dt_flag==True].zonal_velocity.size.compute()

0

In [10]:
time = df.index.values.compute()

In [12]:
time = np.unique(time)

In [49]:
df = df.reset_index().persist()

In [52]:
df[df.time==time[::6]]

ValueError: Metadata inference failed in `eq`.

Original error is below:
------------------------
ValueError('Lengths must match to compare', (2,), (1441,))

Traceback:
---------
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask/dataframe/utils.py", line 175, in raise_on_meta_error
    yield
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask/dataframe/core.py", line 5172, in elemwise
    meta = partial_by_order(*parts, function=op, other=other)
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/dask/utils.py", line 1107, in partial_by_order
    return function(*args2, **kwargs)
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/pandas/core/ops/common.py", line 65, in new_method
    return method(self, other)
  File "/home1/datahome/zcaspar/miniconda3/envs/croco/lib/python3.8/site-packages/pandas/core/ops/__init__.py", line 370, in wrapper
    res_values

In [47]:
df.groupby(df.index).resample('6H')

AttributeError: 'Column not found: resample'

In [46]:
df_sub = df.resample(np.timedelta64(6,'h')).asfreq()#.persist()

ValueError: Invalid frequency: 6 hours

In [39]:
df_sub.persist()

AttributeError: 'Resampler' object has no attribute 'persist'

In [33]:
np.unique(df_sub.index.values.compute()).size
df_sub.index.values.compute().shape

(101996318,)

In [34]:
df.index.values.compute().shape

(611978031,)

In [53]:
#client.restart()
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
